<a href="https://colab.research.google.com/github/rohandawar/CarBrandClassificationinDeep-Learning/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link for how to copy kaggle data into google colab:
https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a

another link:

https://pub.towardsai.net/car-brand-classification-in-deep-learning-with-python-d08c54add941

In [2]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

In [4]:
#check the present working directory
!pwd

/content


In [6]:
#change to the directory where the file is present
%cd /content/gdrive/My Drive/Kaggle
!pwd

/content/gdrive/My Drive/Kaggle
/content/gdrive/My Drive/Kaggle


In [7]:
#copy and paste the API command from kaggle to collab notebook
!kaggle datasets download -d ritesh2000/car-brand-images-dataset

  0% 0.00/1.06M [00:00<?, ?B/s]
100% 1.06M/1.06M [00:00<00:00, 35.7MB/s]


In [8]:
#Check the content of the diectory
!ls

car-brand-images-dataset.zip  kaggle.json


In [ ]:
#unzip the data and remove the ZIP file
!unzip \*.zip && rm *.zip

In [10]:
# Importing the important Libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

#import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt

from glob import glob

In [12]:
#resizing all the images to 224, 224
IMAGE_SIZE = [224, 224]

train_path = 'Train'
test_path = 'Test'

In [13]:
# Use imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet',
                  include_top = False)

94773248/94765736 [==============================] - 1s 0us/step


In [17]:
#don't train existing weights

for layer in resnet.layers:
  layer.trainable = False

#no of output classes
folders = glob('Train/*')

#our Layers
x = Flatten()(resnet.output)

prediction = Dense(len(folders), activation='softmax')(x)

#creating object model

model = Model(inputs= resnet.input, outputs= prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [18]:
#compile the model

model.compile(loss='categorical_crossentropy', optimizer= 'adam', 
              metrics = ['accuracy'])


#Using the Image Data Generator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('Train', 
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

r = model.fit_generator(training_set, validation_data=test_set,
                        epochs = 50,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
2/2 [==============================] - 21s 13s/step - loss: 4.7555 - accuracy: 0.2604 - val_loss: 9.9228 - val_accuracy: 0.5172
Epoch 2/50
2/2 [==============================] - 17s 12s/step - loss: 9.6457 - accuracy: 0.3229 - val_loss: 4.7283 - val_accuracy: 0.1552
Epoch 3/50
2/2 [==============================] - 17s 12s/step - loss: 4.3895 - accuracy: 0.3646 - val_loss: 11.2184 - val_accuracy: 0.3276
Epoch 4/50
2/2 [==============================] - 17s 12s/step - loss: 7.5509 - accuracy: 0.3854 - val_loss: 8.8176 - val_accuracy: 0.1897
Epoch 5/50
2/2 [==============================] - 17s 12s/step - loss: 5.6034 - accuracy: 0.4167 - val_loss: 5.0313 - val_accuracy: 0.1552
Epoch 6/50
2/2 [==============================] - 17s 12s/step - loss: 2.2795 - accuracy: 0.5104 - val_loss: 2.0471 - val_accuracy: 0.5690
Epoch 7/50
2/2 [==============================] - 17s 12s/step - loss: 2.7597 - accuracy: 0.4167 - val_loss: 2.6858 - val_accuracy: 0.5690
Epoch 8/50
2/2 [==========